In [1]:
!pip install datasets -q

     |████████████████████████████████| 266kB 8.4MB/s 
     |████████████████████████████████| 51kB 7.8MB/s 
     |████████████████████████████████| 122kB 49.8MB/s 
     |████████████████████████████████| 245kB 41.8MB/s 


In [2]:
from datasets import load_dataset
dataset = load_dataset("squad_v2")

Dataset squad_v2 downloaded and prepared to /root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/de2e67b822b2ef3f4b137148d0758f48075e3892c359c50271ef6c9add7e794a. Subsequent calls will reuse this data.


In [3]:
import pandas as pd
train = dataset["train"]
valid = dataset["validation"]
train_df = pd.DataFrame(train)
valid_df = pd.DataFrame(valid)

In [5]:
# df = train_df[["context", "question"]]
# train_data =

In [6]:
df = pd.DataFrame(columns=["input_text", "target_text"])

df["input_text"] = train_df["context"]
df["target_text"] = train_df["question"]
df["prefix"] = "ask_question"

In [7]:
df[:2]

,input_text,target_text,prefix
0,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce start becoming popular?,ask_question
1,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What areas did Beyonce compete in when she was...,ask_question


In [8]:
train_df = df
df = pd.DataFrame(columns=["input_text", "target_text"])

df["input_text"] = valid_df["context"]
df["target_text"] = valid_df["question"]
df["prefix"] = "ask_question"
valid_df = df

In [9]:
!pip install simpletransformers -q

     |████████████████████████████████| 225kB 9.3MB/s 
     |████████████████████████████████| 3.3MB 17.4MB/s 
     |████████████████████████████████| 7.8MB 45.9MB/s 
     |████████████████████████████████| 1.2MB 44.3MB/s 
     |████████████████████████████████| 133kB 49.2MB/s 
     |████████████████████████████████| 81kB 11.3MB/s 
     |████████████████████████████████| 2.5MB 47.1MB/s 
     |████████████████████████████████| 51kB 8.3MB/s 
     |████████████████████████████████| 1.8MB 35.2MB/s 
     |████████████████████████████████| 112kB 56.6MB/s 
     |████████████████████████████████| 81kB 11.0MB/s 
     |████████████████████████████████| 4.2MB 41.4MB/s 
     |████████████████████████████████| 174kB 59.8MB/s 
     |████████████████████████████████| 901kB 32.7MB/s 
     |████████████████████████████████| 143kB 50.6MB/s 
     |████████████████████████████████| 102kB 15.5MB/s 
     |████████████████████████████████| 122kB 57.1MB/s 
     |████████████████████████████████| 71kB 12.3MB/s

In [10]:
from simpletransformers.t5 import T5Model

In [ ]:
model_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "max_seq_length": 128,
    "train_batch_size": 8,
    "num_train_epochs": 1,
    "save_eval_checkpoints": True,
    "save_steps": -1,
    "use_multiprocessing": False,
    "evaluate_during_training": True,
    "evaluate_during_training_steps": 15000,
    "evaluate_during_training_verbose": True,
    "fp16": False
    
}

model = T5Model("t5", "t5-large", args=model_args, use_cuda= True)

model.train_model(train_df, eval_data=valid_df)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3282: FutureWarning: `prepare_seq2seq_batch` is deprecated and will be removed in version 5 of 🤗 Transformers. Use the regular `__call__` method to prepare your inputs and the tokenizer under the `with_target_tokenizer` context manager to prepare your targets. See the documentation of your specific tokenizer for more details
  FutureWarning,



Using Adafactor for T5


In [1]:
import pprint
preds = model.predict(
    ["ask_question: " + description for description in valid_df["input_text"].tolist()]
)
questions = valid_df["target_text"].tolist()
pprint(valid_df[0]["input_text"])
pprint(preds[0])